In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-dataset/Covid Data.csv


The dataset column description has been provided and i attached it for convenience purposes.

sex: 1 for female and 2 for male.
age: of the patient.
classification: covid test findings. Values 1-3 mean that the patient was diagnosed with covid in different
degrees. 4 or higher means that the patient is not a carrier of covid or that the test is inconclusive.

patient type: type of care the patient received in the unit. 1 for returned home and 2 for hospitalization.

pneumonia: whether the patient already have air sacs inflammation or not.

pregnancy: whether the patient is pregnant or not.

diabetes: whether the patient has diabetes or not.

copd: Indicates whether the patient has Chronic obstructive pulmonary disease or not.

asthma: whether the patient has asthma or not.

inmsupr: whether the patient is immunosuppressed or not.

hypertension: whether the patient has hypertension or not.

cardiovascular: whether the patient has heart or blood vessels related disease.

renal chronic: whether the patient has chronic renal disease or not.

other disease: whether the patient has other disease or not.

obesity: whether the patient is obese or not.

tobacco: whether the patient is a tobacco user.

usmr: Indicates whether the patient treated medical units of the first, second or third level.

medical unit: type of institution of the National Health System that provided the care.

intubed: whether the patient was connected to the ventilator.

icu: Indicates whether the patient had been admitted to an Intensive Care Unit.

date died: If the patient died indicate the date of death, and 9999-99-99 otherwise.

Import the data from the local repository. And inspect the first rows.  

In [2]:
covid_data = pd.read_csv("/kaggle/input/covid19-dataset/Covid Data.csv")

# Inspect the first elements
print(covid_data.head())

# List the column names
print(covid_data.columns)

   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2             1    1             1  03/05/2020       97          1   
1      2             1    2             1  03/06/2020       97          1   
2      2             1    2             2  09/06/2020        1          2   
3      2             1    1             1  12/06/2020       97          2   
4      2             1    2             1  21/06/2020       97          2   

   AGE  PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HIPERTENSION  OTHER_DISEASE  \
0   65         2         2  ...       2        2             1              2   
1   72        97         2  ...       2        2             1              2   
2   55        97         1  ...       2        2             2              2   
3   53         2         2  ...       2        2             2              2   
4   68        97         1  ...       2        2             1              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO

There seem to be some spelling errors in the name of the columns. I changed those in order to provide more consistency with the list of the column names provided by the author

In [3]:
covid_data = covid_data.rename(columns = {"USMER" : "USMR","HIPERTENSION": "HYPERTENSION" })

covid_data.columns

Index(['USMR', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'DATE_DIED', 'INTUBED',
       'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HYPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU'],
      dtype='object')

Next steps

1) replace missing values
2) check for missing values in main outcome variable date_died
3) change date of death to binary format (1: has died, 2: alive)
4) change classification_final to binary (1: has covid, 2: does not have covid)

In [4]:
# 1. replace missing values
covid_data = covid_data.replace(97, np.NaN)
covid_data = covid_data.replace(99, np.NaN)

# 2. check for missing values in date_died

print("Sum of NA values in the datei_died column is: ", covid_data['DATE_DIED'].isnull().sum())

# 3. change data of death to binary

covid_data.loc[covid_data.DATE_DIED != "9999-99-99" ,'DATE_DIED'] = 1
covid_data.loc[covid_data.DATE_DIED == "9999-99-99" ,'DATE_DIED'] = 2

# 4. change classfication_final to binary

covid_data.loc[covid_data.CLASIFFICATION_FINAL <= 3, 'CLASIFFICATION_FINAL'] = 1
covid_data.loc[covid_data.CLASIFFICATION_FINAL > 3, 'CLASIFFICATION_FINAL'] = 2



Sum of NA values in the datei_died column is:  0


In [5]:
print(covid_data.head())

   USMR  MEDICAL_UNIT  SEX  PATIENT_TYPE DATE_DIED  INTUBED  PNEUMONIA   AGE  \
0     2             1    1             1         1      NaN        1.0  65.0   
1     2             1    2             1         1      NaN        1.0  72.0   
2     2             1    2             2         1      1.0        2.0  55.0   
3     2             1    1             1         1      NaN        2.0  53.0   
4     2             1    2             1         1      NaN        2.0  68.0   

   PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HYPERTENSION  OTHER_DISEASE  \
0       2.0         2  ...       2        2             1              2   
1       NaN         2  ...       2        2             1              2   
2       NaN         1  ...       2        2             2              2   
3       2.0         2  ...       2        2             2              2   
4       NaN         1  ...       2        2             1              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO  CLASIFFICA

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
#print(covid_data.head())

# IMPORTANT TO DO: check if there are NAs in this column and rename column




In [7]:
print(covid_data.iloc[1:10,])

   USMR  MEDICAL_UNIT  SEX  PATIENT_TYPE DATE_DIED  INTUBED  PNEUMONIA   AGE  \
1     2             1    2             1         1      NaN        1.0  72.0   
2     2             1    2             2         1      1.0        2.0  55.0   
3     2             1    1             1         1      NaN        2.0  53.0   
4     2             1    2             1         1      NaN        2.0  68.0   
5     2             1    1             2         2      2.0        1.0  40.0   
6     2             1    1             1         2      NaN        2.0  64.0   
7     2             1    1             1         2      NaN        1.0  64.0   
8     2             1    1             2         2      2.0        2.0  37.0   
9     2             1    1             2         2      2.0        2.0  25.0   

   PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HYPERTENSION  OTHER_DISEASE  \
1       NaN         2  ...       2        2             1              2   
2       NaN         1  ...       2        2    

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
